In [237]:
# Initialize Otter
import otter
grader = otter.Notebook("ps9.ipynb")


# PS9: Map and reduce with PySpark

In [238]:
import pyspark
spark = pyspark.sql.SparkSession.builder.\
    appName('ps9').\
    config('spark.ui.showConsoleProgress', False).\
    getOrCreate()
sc = spark.sparkContext


# Problem 1

This problem concerns counting words in text using `PySpark`.

In [239]:
simple_text_rdd = sc.textFile('simple.txt')
simple_text_rdd.collect()


['Raining cats and dogs',
 'Cats eat mice, mice eat cheese',
 'Mice and cats and dogs have fleas']

In [240]:
type(simple_text_rdd)


pyspark.rdd.RDD

## Part (a)
Write a function, called `count_words_per_line(text_rdd)`, which counts how many time each word appears in each line. The input `text_rdd` is a Resilient Distributed Dataset (`RDD`) of a text file, and the output will be another `RDD`.
- You should be able to perform this using solely `.map` operations on `text_rdd`.
- Your script should strip commas, colons, semicolons and periods, but leave other punctuation in place.
- Your script should ignore case.
- The output should be an `RDD` of `Counter` objects. When `.collect()` is called, the output should have the same length as the number of lines in the text.

In [241]:
from collections import Counter


In [242]:
def count_words_per_line(text: pyspark.RDD):
    return text.map(str.lower)\
        .map(lambda line: line.replace(",", "").replace(":", "").replace(";", "").replace(".", "").split(" "))\
        .map(lambda line: Counter(line))


In [243]:
count_words_per_line(simple_text_rdd).collect()


[Counter({'raining': 1, 'cats': 1, 'and': 1, 'dogs': 1}),
 Counter({'cats': 1, 'eat': 2, 'mice': 2, 'cheese': 1}),
 Counter({'mice': 1, 'and': 2, 'cats': 1, 'dogs': 1, 'have': 1, 'fleas': 1})]

In [244]:
grader.check("q1a")


q1a results: All test cases passed!

## Part (b)
Using your answer from part (a), write a function called `count_words` that returns a `Counter` of the words in the input text.
(Hint: use the `.reduce()` or `.fold()` method)

In [245]:
from operator import add


In [246]:
def count_words(text: pyspark.RDD):
    return count_words_per_line(text).reduce(add)


In [247]:
count_words(simple_text_rdd)


Counter({'raining': 1,
         'cats': 3,
         'and': 3,
         'dogs': 2,
         'eat': 2,
         'mice': 3,
         'cheese': 1,
         'have': 1,
         'fleas': 1})

In [248]:
grader.check("q1b")


q1b results: All test cases passed!

## Part (c)

Write a function called `counter_to_rdd` that takes a Counter object and returns an RDD object such that each element is `(word, log(count))`, where `log()` is the natural logarithm, and sorted in descending order by `log(count)`.

In [249]:
import math


def counter_to_rdd(word_count):

    # count in descending order
    dic = word_count.most_common()

    # creat a RDD object
    rdd = sc.parallelize(dic)

    # map
    res = rdd.map(lambda line: (line[0], math.log(line[1])))

    return res


In [250]:
darwin_rdd = sc.textFile('darwin.txt')
darwin_rdd.first()


'The Project Gutenberg eBook of On the Origin of Species, by Charles Darwin'

In [251]:
darwin_word_count = count_words(darwin_rdd)
counter_to_rdd(darwin_word_count).collect()


[('the', 9.590282522853261),
 ('of', 9.263975921142093),
 ('and', 8.678631537293768),
 ('in', 8.6020856584342),
 ('to', 8.481358738407021),
 ('a', 8.1101268019411),
 ('that', 7.9102237070973445),
 ('', 7.787382026484701),
 ('as', 7.699842407396986),
 ('have', 7.6586995582682995),
 ('be', 7.656810091480378),
 ('is', 7.641564441260972),
 ('on', 7.554858521040676),
 ('by', 7.520234556474628),
 ('species', 7.509335266016592),
 ('which', 7.487173694213739),
 ('or', 7.450660796211539),
 ('are', 7.419380582918692),
 ('it', 7.3343293503005365),
 ('with', 7.315218389752975),
 ('for', 7.29505641646263),
 ('from', 7.267525427828172),
 ('this', 7.217443431696533),
 ('we', 7.189922170745808),
 ('been', 7.1252830915107115),
 ('not', 7.068172000388042),
 ('but', 7.026426808699636),
 ('same', 6.964135612418245),
 ('their', 6.903747257584598),
 ('other', 6.85751406254539),
 ('i', 6.84587987526405),
 ('at', 6.75110146893676),
 ('has', 6.71295620067707),
 ('some', 6.693323668269949),
 ('one', 6.678342114

In [252]:
grader.check("q1c")


q1c results: All test cases passed!

<!-- BEGIN QUESTION -->

## Part (d)

Collect the output of the sorted log word counts and plot it using `matplotlib`.

Zipf’s law
states, roughly, that if one plots word frequency against frequency rank
(i.e., most frequent word, second most frequent word, etc.), the resulting line is (approximately) linear on a log-log scale. Using the information in darwin word counts.txt,
make a plot of word frequency as a function of word rank on a log-log scale for all
words in the file
`darwin.txt` 

How “Zipfian” does the resulting plot look? Write an answer in the box below. (It suffices for you to state whether or
not your plot looks approximately like a line)

The resulting line is (approximately) linear on a log-log scale. 

In [253]:
from matplotlib import pyplot as plt
darwin_word_count = count_words(darwin_rdd)
lst = list(enumerate(counter_to_rdd(darwin_word_count).collect(), 1))
x = [math.log(i[0]) for i in lst]
y = [i[1][1] for i in lst]
fig = plt.plot
fig(x, y, "o-")
plt.draw()
plt.show()


<!-- END QUESTION -->

# Problem 2
The performance gains from PySpark come from using its SQL-like syntax on tabular data.

## Part (a)
Write a function called `read_population_df` that reads in `populations_small.txt` as a PySpark DataFrame. The result should have two columns labelled `id` and `value`. 

Note: `id` should be `LongType` and `value` should be `DoubleType`.

In [254]:
from pyspark.sql import types as T
from pyspark.sql import functions as F


In [255]:
def get_csv_schema(*args):
    return T.StructType([
        T.StructField(*arg)
        for arg in args
    ])


In [256]:
def read_population_df(filename: str) -> pyspark.sql.DataFrame:

    # initialize data
    df = spark.read.text(filename)

    # split column and rename
    df = df.select(F.split(df.value, " ")).rdd.flatMap(
        lambda x: x).toDF(schema=["id", "value"])

    # cast types
    df = df.withColumn("id", df["id"].cast(T.LongType())).withColumn(
        "value", df["value"].cast(T.DoubleType()))

    return df


In [257]:
grader.check("q2a")


q2a results: All test cases passed!

## Part (b)

Write a function called `pop_mean_stddev` that takes a DataFrame with two columns, `id` and `value`,
and returns a data-frame where each row indicates mean and standard deviation of `value` for a particular value of `id`.

The resulting DataFrame should have three columns (`id`, `mean` and `stddev`) and should be sorted in ascending order by `id`.

In [258]:
def pop_mean_stddev(pop_df):

    # group by id and calculate mean
    df1 = pop_df.groupBy("id").agg(F.mean("value").alias("mean"))

    # group by id and calculate stddev
    df2 = pop_df.groupBy("id").agg(F.stddev("value").alias("stddev"))

    # merge the two dataframes
    new_df = df1.join(df2, on=["id"], how="left").sort("id")

    return new_df


In [259]:
grader.check("q2b")


q2b results: All test cases passed!

# Problem 3

Recall the `fb.txt.gz` text file. You are given a PySpark Dataframe `fb_df` with one column called "value", which is a string. The first number is the user id, and the following number are their friends.

In [260]:
import pyspark.sql.functions as F
fb_df = spark.read.csv("fb.txt.gz")
fb_df.head(10)


[Row(_c0='0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 

## Part (a)

Write a function called parse_friends which transforms `fb_df` into a PySpark dataframe with two columns:
 - `id`: String indicating the friend
 - `friends`: An array column where each entry is the friends of `id` whose ids come after `id`.
 

```
>>> friends = parse_friends(fb_df)
>>> friends.take(2)
[Row(id='0', friends=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '160', '161', '162', '163', '164', '165', '166', '167', '168', '169', '170', '171', '172', '173', '174', '175', '176', '177', '178', '179', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '190', '191', '192', '193', '194', '195', '196', '197', '198', '199', '200', '201', '202', '203', '204', '205', '206', '207', '208', '209', '210', '211', '212', '213', '214', '215', '216', '217', '218', '219', '220', '221', '222', '223', '224', '225', '226', '227', '228', '229', '230', '231', '232', '233', '234', '235', '236', '237', '238', '239', '240', '241', '242', '243', '244', '245', '246', '247', '248', '249', '250', '251', '252', '253', '254', '255', '256', '257', '258', '259', '260', '261', '262', '263', '264', '265', '266', '267', '268', '269', '270', '271', '272', '273', '274', '275', '276', '277', '278', '279', '280', '281', '282', '283', '284', '285', '286', '287', '288', '289', '290', '291', '292', '293', '294', '295', '296', '297', '298', '299', '300', '301', '302', '303', '304', '305', '306', '307', '308', '309', '310', '311', '312', '313', '314', '315', '316', '317', '318', '319', '320', '321', '322', '323', '324', '325', '326', '327', '328', '329', '330', '331', '332', '333', '334', '335', '336', '337', '338', '339', '340', '341', '342', '343', '344', '345', '346', '347']),
 Row(id='1', friends=['48', '53', '54', '73', '88', '92', '119', '126', '133', '194', '236', '280', '299', '315', '322', '346'])]
 ```
 `

In [261]:
def parse_friends(fb_df):

    # read csv
    fb_df = spark.read.csv("fb.txt.gz")

    # split id out and remove a space in value
    split_col = F.split(fb_df["_c0"], " ")
    fb_df = fb_df.withColumn("id", split_col[0]).withColumn(
        "friends", F.expr("substring(_c0, 1, length(_c0))")).drop("_c0")

    # friends: string-> array<strings>
    fb_df = fb_df.withColumn("friends", F.split(F.col("friends"), " "))

    # remove the first element which is id in the friends
    fb_df = fb_df.withColumn("friends", F.expr(
        "slice(friends, 2, SIZE(friends))"))

    return fb_df


In [262]:
grader.check("q3a")


q3a results: All test cases passed!

## Part (b)

Write a function called `get_friend_pairs(fb_df)` which finds all pairs of friends in `fb_df`.

In [263]:
def get_friend_pairs(fb_df):

    # initialize dataframe
    fb_df = parse_friends(fb_df)
    df = fb_df

    # explode friends
    df = df.select(df.id, F.explode(df.friends).alias("friend2"))

    # rename id
    df = df.withColumnRenamed("id", "friend1")

    return df


+-------+-------+
|friend1|friend2|
+-------+-------+
|      0|      1|
|      0|      2|
|      0|      3|
|      0|      4|
|      0|      5|
|      0|      6|
|      0|      7|
|      0|      8|
|      0|      9|
|      0|     10|
|      0|     11|
|      0|     12|
|      0|     13|
|      0|     14|
|      0|     15|
|      0|     16|
|      0|     17|
|      0|     18|
|      0|     19|
|      0|     20|
+-------+-------+
only showing top 20 rows



In [264]:
grader.check("q3b")


q3b results: All test cases passed!

## Part (c)
Write a function called `find_triangles(fb_df)` which finds the number of triangles in `fb_df`.

```
>>> find_triangles(fb_df).take(6)
[Row(friend1='0', friend2='1', friend3='48'),
 Row(friend1='0', friend2='1', friend3='53'),
 Row(friend1='0', friend2='1', friend3='54'),
 Row(friend1='0', friend2='1', friend3='73'),
 Row(friend1='0', friend2='1', friend3='88'),
 Row(friend1='0', friend2='1', friend3='92')]
```

In [265]:
def find_triangles(fb_df):

    # initialize data and change datatype
    df = get_friend_pairs(fb_df)
    df = df\
        .withColumn("friend1", df["friend1"].cast("int"))\
        .withColumn("friend2", df["friend2"].cast("int"))

    # copy dataframe
    df1 = df.withColumnRenamed(
        "friend1", "a_people").withColumnRenamed("friend2", "a_friend")
    df2 = df.withColumnRenamed(
        "friend1", "b_people").withColumnRenamed("friend2", "b_friend")

    # merge
    merged_df = df1.join(df2)\
        .where(df1["a_friend"] == df2["b_friend"])\
        .where(df1["a_people"] < df2["b_people"])\
        .select(["a_people", "b_people", "a_friend"])\
        .sort(["a_people", "b_people", "a_friend"], ascending=True)\
        .withColumnRenamed("a_people", "friend1").withColumnRenamed("b_people", "friend2").withColumnRenamed("a_friend", "friend3")

    # connect friendship between friend1 and friend2
    merged_df = merged_df.join(df, ["friend1", "friend2"], "left_semi")

    # change datatype
    merged_df = merged_df.withColumn("friend1", merged_df["friend1"].cast(T.StringType())).withColumn(
        "friend2", merged_df["friend2"].cast(T.StringType())).withColumn("friend3", merged_df["friend3"].cast(T.StringType()))

    return merged_df


In [266]:
grader.check("q3c")

q3c results: All test cases passed!

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [267]:
grader.check_all()


q1a results: All test cases passed!

q1b results: All test cases passed!

q1c results: All test cases passed!

q2a results: All test cases passed!

q2b results: All test cases passed!

q3a results: All test cases passed!

q3b results: All test cases passed!

q3c results: All test cases passed!

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Upload this .zip file to Gradescope for grading.

In [268]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)


22/03/26 12:19:05 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 303732 ms exceeds timeout 120000 ms
22/03/26 12:19:05 WARN SparkContext: Killing executors is not supported by current scheduler.
